In [11]:
from IPython.display import display, HTML
display(HTML('<style> .container{width : 100%} </style>'))

In [4]:
import subprocess
import os
import json
import requests
import math
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
url = input('Enter animefreak.site link here: ') #https://animefreak.site/anime/sword-art-online-alicization-war-of-underworld

In [ ]:
from_ep = int(input('Download from episode no: '))

In [ ]:
max_download = int(input('Max download size: '))

In [4]:
class json_io:
    
    def read(name):
        with open(name, 'r') as f:
            data = json.load(f)
            f.close()
        return data
            
    def write(name, mode, fdata):
        if mode == 'a':
            data = json_io.read(name)
            data.update(fdata)
        else:
            data = fdata
        with open(name, 'w') as f:
            s = json.dumps(data, indent = 4)
            print(s, file = f)

In [5]:
class Anime:
    
    def __init__(self, url):
        self.url = url
        self.link_name = url.split('/')[-1]
        self.name = self.link_name.replace('-', ' ').title()
        self.episode = self.extract_ep()
        
    def extract_ep(self):
        
        if 'animefreak.site' in  self.url:
            episode = {}
            t_url = f'https://animefreak.site/watch/{self.link_name}-episode-1'
            t_r = requests.get(t_url)
            t_soup = BeautifulSoup(t_r.content, 'html.parser')
            for i in t_soup.find_all('a', {'class' : ['ssl-item ep-item', 'ssl-item ep-item active']}):
                episode[i.get('title')] = f"https://animefreak.site{i.get('href')}"
            episode = dict(reversed(episode.items()))

            return episode
        
        if 'gogoanime2.org' in  self.url:
            episode = {}
            t_r = requests.get(self.url)
            t_soup = BeautifulSoup(t_r.content, 'html.parser')
            for i in t_soup.find_all('div', {'class':'anime_video_body'})[0].find_all('a'):
                episode[i.find('div').text.replace('  ',' ')] = f"https://ww3.gogoanime2.org{i.get('href')}"

            return episode

In [73]:
class browser:
    
    __path = 'C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe'
    __headers =  {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Accept-Language' : 'en-GB,en;q=0.5',
    'Referer' : 'https:google.com',
    'DNT' : '1'
    }
    
    def initiate(url):
        
        options = Options()
        options.binary_location = browser.__path
#         options.add_argument('--user-data-dir=~/.e2e-chrome-profile')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        caps = DesiredCapabilities.CHROME
        caps['goog:loggingPrefs'] = {'performance' : 'ALL'}
        
        driver = webdriver.Chrome(service = Service('backend/chromedriver.exe'),desired_capabilities = caps, options = options)
        
        stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )
        
#         driver.request_interceptor = browser.__headers
        
        driver.get(url)
        sleep(20)
        perf = driver.get_log('performance')
        driver.close()
        
        return perf

In [7]:
def extract_m3u8(y):
    for a in y:
        if '.m3u8' in str(a):
            for b in str(a).split(','):
                if '.m3u8' in b:
                    link = b[7:-2]
                    break
            break
    return link

In [8]:
def cli_command(li, name):
    
    link = extract_m3u8(li)
    
    curr_dir = os.getcwd()
    ffmpeg_path = curr_dir + '\\backend\\ffmpeg\\bin\\ffmpeg'
    op = curr_dir + '\\' + output_path + '\\' + name + '.mkv'

    string = f"{ffmpeg_path} -i {link} -c copy -bsf:a aac_adtstoasc {op}"
    
    return string

In [ ]:
anime = Anime(url)

log = {anime.name : { 'url' : anime.url,
              'episode': anime.episode}}

json_io.write(f'{anime.name}.json', 'w', log)

In [ ]:
output_path = f"Download\\{anime.name.replace(' ', '_')}"
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [ ]:
# for i in log:
#     for j in list(log[i]['episode'])[from_ep:]:

#         name = j.split()[-1]
#         print(name)
#         web_link = log[i]['episode'][j]

#         y = browser.initiate(web_link)

#         cmd = cli_command(y, name)

#         os.system(cmd)
        
#         log[i]['episode'].pop(j)

#         json_io.write(f'{anime.name}.json', 'w', log)

#     os.remove(f'{anime.name}.json')

In [24]:
for i in log:
    
    ep_li = list(log[i]['episode'])[from_ep:]
    batch_size = math.ceil(len(ep_li) / max_download) 
    batch = np.array_split(ep_li, batch_size)
    
    for n in batch:
        
        li = []
        
        try:
            for j in n:
                name = j.split()[-1]
                web_link = log[i]['episode'][j]
                y = browser.initiate(web_link)
                cmd = cli_command(y, name)
                li.append((j, name, cmd))
        except:
            pass

        for j in li[1:]:
            os.system(f'start cmd /k {j[2]}')
            log[i]['episode'].pop(j[0])
            
        os.system(f'start cmd /k {li[0][2]}')
        
        log[i]['episode'].pop(li[0][0])
          
        json_io.write(f'{anime.name}.json', 'w', log)

    os.remove(f'{anime.name}.json')   

Enter animefreak.site link here: https://animefreak.site/anime/overlord-ii
Download from episode no: 0
Max download size: 3


C:\Users\caaka\AppData\Local\Temp\ipykernel_16984\3251724408.py:88: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = 'backend/chromedriver.exe',desired_capabilities = caps, options = options)


UnboundLocalError: local variable 'link' referenced before assignment

In [19]:
import undetected_chromedriver as uc

In [32]:
from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service

In [74]:
url = 'https://ww3.gogoanime2.org/watch/isekai-meikyuu-de-harem-wo-uncensored/1'

In [75]:
y = browser.initiate(url)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A378B3+2193587]
	Ordinal0 [0x009D0681+1771137]
	Ordinal0 [0x008E4070+802928]
	Ordinal0 [0x008D9AB2+760498]
	Ordinal0 [0x008CC440+705600]
	Ordinal0 [0x00933BC0+1129408]
	Ordinal0 [0x0092D773+1103731]
	Ordinal0 [0x009077E0+948192]
	Ordinal0 [0x009086E6+952038]
	GetHandleVerifier [0x00CE0CB2+2738370]
	GetHandleVerifier [0x00CD21B8+2678216]
	GetHandleVerifier [0x00AC17AA+512954]
	GetHandleVerifier [0x00AC0856+509030]
	Ordinal0 [0x009D743B+1799227]
	Ordinal0 [0x009DBB68+1817448]
	Ordinal0 [0x009DBC55+1817685]
	Ordinal0 [0x009E5230+1856048]
	BaseThreadInitThunk [0x76A96739+25]
	RtlGetFullPathName_UEx [0x77D48FD2+1218]
	RtlGetFullPathName_UEx [0x77D48F9D+1165]


In [76]:
path = 'C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe'
headers =  {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Accept-Language' : 'en-GB,en;q=0.5',
    'Referer' : 'https:google.com',
    'DNT' : '1'
}
options = Options()
options.binary_location = path
options.add_argument('proxy-server=106.122.8.54:3128')
options.add_argument('--user-data-dir=C:/Users/caaka/AppData/Local/BraveSoftware/Brave-Browser/User Data/Default')
driver2 = uc.Chrome(options = options)
driver2.request_interceptor = headers
driver2.get(url)